In [ ]:
import pandas as pd
from pathlib import Path

dir = Path('/Users/kamma/Downloads')
base_fname = 'AIS_2022_01_01'

pq_fname = (dir / base_fname).with_suffix('.parquet')
if pq_fname.exists():
    print("Loading parquet")
    df = pd.read_parquet(pq_fname)
else:
    if (fname := (dir / base_fname).with_suffix('.csv.gz')).exists():
        print("Loading csv.gz")
        df = pd.read_csv(fname)
    elif (fname := (dir / base_fname).with_suffix('.zip')).exists():
        print("Loading zip")
        df = pd.read_csv(fname)
    df = df.assign(BaseDateTime=pd.to_datetime(df.BaseDateTime))
    df.to_parquet(pq_fname)
df

In [ ]:
import numpy as np
from numba import jit

@jit(nopython=True)
def haversine(lon1, lat1, lon2, lat2, earth_radius=6371.009):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)
    
    # lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = earth_radius * c
    return km

In [ ]:
def get_running_intervals(vdf, speed_threshold, window_size=5):
    low_speed = vdf["speed"] < speed_threshold
    count_speed = low_speed.rolling(window_size, center=True).sum().fillna(method='ffill').fillna(method='bfill')
    zeros = count_speed[count_speed < 1].index.to_series()
    fives = count_speed[count_speed > window_size-1].index.to_series()
    zeros.iloc[:] = False
    fives.iloc[:] = True
    mix = pd.concat([zeros, fives]).sort_index()
    mismatch = (mix != mix.shift().fillna(mix))
    if len(mismatch) > 1:
        mismatch_back = mismatch.shift(-1).fillna(False)
        back_matches = mix[mismatch_back & mix].index
        forward_matches = mix[mismatch & mix].index
    else:
        back_matches = pd.Series(dtype='int').index
        forward_matches = pd.Series(dtype='int').index

    extras = []
    if len(mix[mismatch]) == 0:
        if len(zeros) > len(fives):
            extras.append(low_speed.iloc[:1].index[0])
            extras.append(low_speed.iloc[-1:].index[0])
    else:
        if mix[mismatch].iloc[0]:
            # False -> True so include 0
            extras.append(low_speed.iloc[:1].index[0])
        if not mix[mismatch].iloc[-1]:
            # True -> False so include -1
            extras.append(low_speed.iloc[-1:].index[0])
    if len(extras) > 0:
        s = pd.concat([pd.Series(extras, extras),back_matches.to_series() + (window_size-1)//2,
                forward_matches.to_series() - (window_size+1)//2]).sort_index()
    else:
        s = pd.concat([back_matches.to_series() + (window_size-1)//2,
                forward_matches.to_series() - (window_size+1)//2]).sort_index()
    intervals = s.values.reshape((len(s)//2,2))
    return intervals

In [ ]:
full_df = df.sort_values(['MMSI','BaseDateTime'])

In [ ]:
from IPython.display import display
from tqdm.auto import tqdm

LAT_COL = "LAT"
LON_COL = "LON"
SPEED_MIN = 0.026 / 60.0 # 26 m/min ~ 1 mph, 0.015 / 60.0 # 15 meters/minute
SPEED_MAX = 8 / 60.0 # 8 km/min ~ 300 mph
TIME_THRESHOLD = pd.Timedelta(minutes=120)
WINDOW_SIZE = 5

cur_id = 0
subtrajectories = []
for mmsi, vdf in tqdm(full_df.groupby('MMSI')):
    vdf = (vdf.drop_duplicates(subset="BaseDateTime", keep="first")
        .reset_index(drop=True)
        .assign(
            dist=lambda df: haversine(df[LON_COL].values, df[LAT_COL].values, 
                            df[LON_COL].shift(1).values, df[LAT_COL].shift(1).values),
            diff_dt=lambda df: df.BaseDateTime.diff(),
            speed=lambda df: df.dist / df.diff_dt.dt.total_seconds()
        )
    )

    # gaps
    gaps = (vdf["diff_dt"] > TIME_THRESHOLD) | (vdf["speed"] > SPEED_MAX)
    last_idx = vdf.iloc[-1:].index[0]+1
    gaps_idx = pd.concat([pd.Series([0],[0]), vdf[gaps].index.to_series(), pd.Series([last_idx],[last_idx])])
    gaps_intervals = pd.DataFrame([gaps_idx, gaps_idx.shift(-1) - 1]).T.iloc[:-1].astype(int)

    for (start, end) in gaps_intervals.itertuples(index=False):
        svdf = vdf.loc[start:end]
        # if mmsi == 270995:
        #     display(svdf.head(5), svdf.tail(5))
        rints = get_running_intervals(svdf, SPEED_MIN, WINDOW_SIZE)
        # if mmsi == 270995:
        #     display(rints)
        for row in rints:
            rvdf = svdf.loc[row[0]:row[1]].assign(SubtrajID=cur_id)
            # rvdf['SubtrajID'] = cur_id
            cur_id += 1
            subtrajectories.append(rvdf)
            # print("GOT RINTS", len(subtrajectories))
len(subtrajectories)

In [ ]:
subtrajs = pd.concat(subtrajectories)
subtrajs[['MMSI','BaseDateTime',LAT_COL,LON_COL,'SOG','COG','Heading','SubtrajID']].to_parquet((dir / (base_fname + '-moving')).with_suffix('.parquet'))


In [ ]:
import pandas as pd

from pathlib import Path

dir = Path('/Users/kamma/Downloads')
base_fname = 'AIS_2022_01_01'

moving_fname = (dir / (base_fname + '-moving')).with_suffix('.parquet')

if not moving_fname.exists():
    raise ValueError(f'"{moving_fname}" not found. Run ais-split-voyages.ipynb')
trajs = pd.read_parquet(moving_fname)
trajs

In [ ]:
vis_df = trajs.groupby('SubtrajID')[['LON','LAT']].apply(lambda df: [df.values.astype('float32')]).str[0].rename("path").to_frame()

In [ ]:
import pydeck as pdk

# def hex_to_rgb(h):
#     h = h.lstrip("#")
#     return tuple(int(h[i : i + 2], 16) for i in (0, 2, 4))

# final_viz_redu["color"] = final_viz_redu["color"].apply(hex_to_rgb)

view_state = pdk.ViewState(longitude=-82.77474, latitude=28.08771, zoom=6)

layer = pdk.Layer(
    type="PathLayer",
    data=vis_df,
    pickable=True,
    # get_color="color",
    get_color=[255,0,0,255],
    width_scale=20,
    width_min_pixels=2,
    get_path="path",
    get_width=5,
    use_binary_transport=False
)

r = pdk.Deck(layers=[layer], initial_view_state=view_state) #, tooltip={"text": "{MMSI}"})
r.to_html()